In [1]:
!pip uninstall -y lightgbm
!apt-get install -y libboost-all-dev
!git clone --recursive https://github.com/Microsoft/LightGBM

Found existing installation: lightgbm 3.3.2
Uninstalling lightgbm-3.3.2:
  Successfully uninstalled lightgbm-3.3.2



The following additional packages will be installed:
  autoconf automake autotools-dev cpp-8 file gcc-8 gcc-8-base gfortran
  gfortran-8 gfortran-9 ibverbs-providers icu-devtools libboost-atomic-dev
  libboost-atomic1.71-dev libboost-atomic1.71.0 libboost-chrono-dev
  libboost-chrono1.71-dev libboost-chrono1.71.0 libboost-container-dev
  libboost-container1.71-dev libboost-container1.71.0 libboost-context-dev
  libboost-context1.71-dev libboost-context1.71.0 libboost-coroutine-dev
  libboost-coroutine1.71-dev libboost-coroutine1.71.0 libboost-date-time-dev
  libboost-date-time1.71-dev libboost-date-time1.71.0 libboost-exception-dev
  libboost-exception1.71-dev libboost-fiber-dev libboost-fiber1.71-dev
  libboost-fiber1.71.0 libboost-graph-dev libboost-graph-parallel-dev
  libboost-graph-parallel1.71-dev libboost-graph-parallel1.71.0
  libboost-graph1.71-dev libboost-gra

In [2]:
%%bash
cd LightGBM
rm -r build
mkdir build
cd build
cmake -DUSE_GPU=1 -DOpenCL_LIBRARY=/usr/local/cuda/lib64/libOpenCL.so -DOpenCL_INCLUDE_DIR=/usr/local/cuda/include/ ..
make -j$(nproc)

-- The C compiler identification is GNU 9.4.0
-- The CXX compiler identification is GNU 9.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Found OpenMP_C: -fopenmp (found version "4.5") 
-- Found OpenMP_CXX: -fopenmp (found version "4.5") 
-- Found OpenMP: TRUE (found version "4.5")  
-- Looking for CL_VERSION_2_2
-- Looking for CL_VERSION_2_2 - not found
-- Looking for CL_VERSION_2_1
-- Looking for CL_VERSION_2_1 - not found
-- Looking for CL_VERSION_2_0
-- Looking for CL_VERSION_2_0 - not found
-- Looking for CL_VERSION_1_2
-- Looking for CL_VERSION_1_2 - found
-- Found OpenCL: /usr/local/cuda/lib64/libOpenCL.so (found vers

rm: cannot remove 'build': No such file or directory


In [3]:
!cd LightGBM/python-package/;python setup.py install --precompile

running install
/opt/conda/lib/python3.7/site-packages/setuptools/command/install.py:37: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  setuptools.SetuptoolsDeprecationWarning,
running build
running build_py
creating build
creating build/lib
creating build/lib/lightgbm
copying lightgbm/__init__.py -> build/lib/lightgbm
copying lightgbm/basic.py -> build/lib/lightgbm
copying lightgbm/libpath.py -> build/lib/lightgbm
copying lightgbm/dask.py -> build/lib/lightgbm
copying lightgbm/plotting.py -> build/lib/lightgbm
copying lightgbm/sklearn.py -> build/lib/lightgbm
copying lightgbm/callback.py -> build/lib/lightgbm
copying lightgbm/compat.py -> build/lib/lightgbm
copying lightgbm/engine.py -> build/lib/lightgbm
running egg_info
creating lightgbm.egg-info
writing lightgbm.egg-info/PKG-INFO
writing dependency_links to lightgbm.egg-info/dependency_links.txt
writing requirements to lightgbm.egg-info/requires.txt
writing top-lev

In [4]:
!mkdir -p /etc/OpenCL/vendors && echo "libnvidia-opencl.so.1" > /etc/OpenCL/vendors/nvidia.icd
!rm -r LightGBM

In [5]:
import os
import pandas as pd
import numpy as np
import lightgbm as lgb
from lightgbm import LGBMRegressor
from lightgbm.callback import early_stopping, log_evaluation
from sklearn.model_selection import KFold, GroupKFold
from tqdm import tqdm
from sklearn.metrics import mean_squared_error
import warnings

import math

In [6]:
pd.set_option('display.width', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
warnings.filterwarnings('ignore')

path = os.getcwd()

In [7]:
train = pd.read_csv('../input/kddbr2022apk/train_sequence_fold.csv')
test = pd.read_csv('../input/kddbr2022apk/test_sequence.csv')
df = pd.concat([train, test])

features_1 = pd.read_csv('../input/kddbr2022apk/complete_04.csv')
features_1 = features_1[["Filename","w_std","w_mean","w_median","w_min","w_max","w_var","n_std","n_mean","n_median","n_min","n_max","n_var","d_std","d_mean","d_median","d_min","d_max","d_var"]]
features_1.columns = ["Filename","_w_std","_w_mean","_w_median","_w_min","_w_max","_w_var","_n_std","_n_mean","_n_median","_n_min","_n_max","_n_var","_d_std","_d_mean","_d_median","_d_min","_d_max","_d_var"]
features_2 = pd.read_csv('../input/kddbr2022apk/complete_05.csv')
features_3 = pd.read_csv('../input/kddbr2022apk/complete_06.csv')
features_4 = pd.read_csv('../input/kddbr2022apk/complete_12.csv')

features = pd.merge(features_1, features_2, on='Filename', how='left')
features = pd.merge(features, features_3, on='Filename', how='left')


mario_features_tr = pd.read_csv('../input/kddbr2022apk/mario_train_vec_v1.csv')
mario_features_ts = pd.read_csv('../input/kddbr2022apk/mario_test_vec_v1.csv')

mario_features = pd.concat([mario_features_tr, mario_features_ts])
mario_features2 = pd.read_csv('../input/kddbr2022apk/vecs_v3_fe.csv')


In [8]:
df = pd.merge(df, features, on='Filename', how='left')
df = pd.merge(df, mario_features, on='Filename', how='left')
df = pd.merge(df, mario_features2, on='Filename', how='left')
df = pd.merge(df, features_4, on='Filename', how='left')

df = df.sort_values(by=['run_id', 'run_seq_id']).reset_index(drop=True)

In [9]:
import gc
del features, mario_features, mario_features2, features_4, features_2, features_3, features_1
gc.collect()

42

In [10]:
df['sqrt_altitude'] = np.sqrt(df['Altitude'])
df['sqrt_delta'] = np.sqrt(df['Delta'])
df['sqrt_altitude_delta'] = np.sqrt(df['Altitude'] + df['Delta'])

df['pow2_altitude'] = (df['Altitude'] ** 2)
df['pow2_delta'] = (df['Delta'] ** 2)
df['pow2_altitude_delta'] = (df['Altitude'] + df['Delta']) ** 2

df['log_Altitude'] = np.log(df['Altitude'])
df['log_Delta'] = np.log(df['Delta'])
df['log_base_05_n_mean'] = np.log(df['base_05_n_mean'])

df['pow13_Altitude'] = df['Altitude'] ** (1/3)
df['pow13_Delta'] = df['Delta'] ** (1/3)
df['pow13_base_05_n_mean'] = df['base_05_n_mean'] ** (1/3)

tgt_columns = [
    'Delta', 'Altitude', 'log_base_05_n_mean', 'pow13_base_05_n_mean',
    'base_05_n_mean', '_n_mean', 'base_05_f05_H_1', 'base_05_n_median', 'base_05_n_max', 'base_05_n_min', 'base_05_f05_H_4', 'x', 'base_05_f05_H_25', 'base_05_f05_H_7', 'x_mean', 'x1_mean', 'x2_mean',
    'base_05_w_mean', '_w_mean', 'base_05_f05_W_21', 'base_05_w_median', 'base_05_w_max', 'base_05_w_min', 'base_05_f05_W_18', 'y', 'base_05_f05_W_6', 'base_05_f05_W_0', 'y_mean', 'y1_mean', 'y2_mean',
    'base_06_n_mean', 'base_06_f06_H_1', 'base_06_n_median', 'base_06_n_max',
    'base_06_w_mean', 'base_06_f06_W_21', 'base_06_w_median', 'base_06_w_max',
    'base_12_x', 'base_12_y', 'base_12_distance'
]

seq_win = [2,3,5,10,15,25,30,50,75]
nxt_sft_win = [1,2,3,4,5,10,15]

for col in tqdm(tgt_columns):
    for win in nxt_sft_win:
        df['shift'+str(win)+'_'+col] = df.groupby('run_id')[col].shift(win).reset_index(0,drop=True)
        df['next'+str(win)+'_'+col] = df.groupby('run_id')[col].shift(-win).reset_index(0,drop=True)
        df['seq_diff_last'+str(win)+'_'+col] = df.groupby('run_id')[col].diff(win).reset_index(0,drop=True)
        df['seq_diff_next'+str(win)+'_'+col] = df.groupby('run_id')[col].diff(-win).reset_index(0,drop=True)
    
    for win in seq_win:
        df['seq_avg_w'+str(win)+'_'+col] = df.groupby('run_id')[col].rolling(win).mean().reset_index(0,drop=True)
        df['seq_std_w'+str(win)+'_'+col] = df.groupby('run_id')[col].rolling(win).std().reset_index(0,drop=True)
        df['seq_max_w'+str(win)+'_'+col] = df.groupby('run_id')[col].rolling(win).max().reset_index(0,drop=True)
        df['seq_min_w'+str(win)+'_'+col] = df.groupby('run_id')[col].rolling(win).min().reset_index(0,drop=True)
    
    
    df['seq_div_last1_'+col] = df[col] / df['shift1_'+col]
    df['seq_div_next1_'+col] = df[col] / df['next1_'+col]
    df['seq_div_last2_'+col] = df[col] / df['shift2_'+col]
    df['seq_div_next2_'+col] = df[col] / df['next2_'+col]
    df['seq_div_last3_'+col] = df[col] / df['shift3_'+col]
    df['seq_div_next3_'+col] = df[col] / df['next3_'+col]
    
    df['seq_min_'+col] = df.groupby('run_id')[col].transform('min')
    df['seq_max_'+col] = df.groupby('run_id')[col].transform('max')
    df['seq_std_'+col] = df.groupby('run_id')[col].transform('std')
    df['seq_avg_'+col] = df.groupby('run_id')[col].transform('mean')

df['r_vetorial_now'] = (df['base_05_n_mean'] ** 2) + (df['base_05_w_mean'] ** 2)
df['shift_r_vetorial'] = (df['shift1_base_05_n_mean'] ** 2) + (df['shift1_base_05_w_mean'] ** 2)
df['shift2_r_vetorial'] = (df['shift2_base_05_n_mean'] ** 2) + (df['shift2_base_05_w_mean'] ** 2)
df['next_r_vetorial'] = (df['next1_base_05_n_mean'] ** 2) + (df['next1_base_05_w_mean'] ** 2)
df['next2_r_vetorial'] = (df['next2_base_05_n_mean'] ** 2) + (df['next2_base_05_w_mean'] ** 2)
df['avg_shift_r_vet'] = (df['r_vetorial_now'] + df['shift_r_vetorial'] + df['shift2_r_vetorial']) / 3
df['avg_next_r_vet'] = (df['r_vetorial_now'] + df['next_r_vetorial'] + df['next2_r_vetorial']) / 3
df['avg_all_r_vet'] = (df['r_vetorial_now'] + df['next_r_vetorial'] + df['next2_r_vetorial'] + df['shift_r_vetorial'] + df['shift2_r_vetorial']) / 5

df['div_r_next2_next'] = df['next2_r_vetorial'] / df['next_r_vetorial']
df['div_r_next_now'] = df['next_r_vetorial'] / df['r_vetorial_now']
df['div_r_now_shift'] = df['r_vetorial_now'] / df['shift_r_vetorial']
df['div_r_shift1_shift2'] = df['shift_r_vetorial'] / df['shift2_r_vetorial']
df['avg_div_r'] = (df['div_r_next2_next'] + df['div_r_next_now'] + df['div_r_now_shift'] + df['div_r_shift1_shift2']) / 4

100%|██████████| 41/41 [06:28<00:00,  9.47s/it]


In [11]:
print(df.shape)

(146262, 3233)


In [12]:
train = df.loc[~df.North.isnull(), :].sort_values(by=['run_id', 'run_seq_id']).reset_index(drop=True)
test = df.loc[df.North.isnull(), :].sort_values(by=['run_id', 'run_seq_id']).reset_index(drop=True)

In [13]:
import gc
del df
gc.collect()

21

In [14]:
train_columns = [col for col in train.columns if col not in ['Filename', 'North', 'East', 'fold', 'run_id', 'run_seq_id']]

In [15]:
n_folds = 5

north_oof_preds = np.zeros(len(train))
east_oof_preds = np.zeros(len(train))

north_test_preds = np.zeros(len(test))
east_test_preds = np.zeros(len(test))

targets = ['North', 'East']

In [16]:
train = train.replace([np.inf, -np.inf], np.nan)
#trn_x = trn_x.fillna(-999)
test = test.replace([np.inf, -np.inf], np.nan)

In [17]:
%%time
for target in targets:
    y = train[target]
    oof_preds = np.zeros(len(train))
    test_preds_fold = np.zeros(len(test))
    score_folds  = []

    # folds = GroupKFold(n_splits=n_folds)
    # for fold_, (trn_, val_) in enumerate(folds.split(y, y, train.run_id)):
    for fold_ in train.fold.sort_values().unique():
        print("Fold: {}".format(fold_))

        trn_ = train.loc[train.fold != fold_].index
        val_ = train.loc[train.fold == fold_].index

        trn_x, trn_y = train.loc[trn_, train_columns], y.loc[trn_]
        val_x, val_y = train.loc[val_, train_columns], y.loc[val_]

#         dtrain = lgb.Dataset(trn_x, trn_y)
#         dvalid = lgb.Dataset(val_x, val_y)
    
        params = {
            'boost': 'gbdt',
            'max_depth': -1,
            'num_leaves': 15,
            'objective': 'regression',
            'min_data_in_leaf': 15,
            'learning_rate': 0.02,
            'feature_fraction': 0.75,
            'bagging_fraction': 0.75,
            'bagging_freq': 100,
            'metric': 'rmse',
            'num_threads': -1,
            'verbosity': -1,
            'seed': 159,
            'device': 'gpu',
            'n_estimators': 10000,
            "gpu_platform_id": 0,
            "gpu_device_id": 0,
            #'tree_learner': 'serial'
        }

        early_stop_cb = early_stopping(stopping_rounds=200)
        log_cb = log_evaluation(period=500, show_stdv=True)
        
        execs = 1
        preds = np.zeros(len(val_x))
        test_preds_exec = np.zeros(len(test))
        
        for p in range(0,execs):
            print("Iteration: {}".format(p+1))
            params['seed'] += p
            model = LGBMRegressor(**params)
            model.fit(trn_x, trn_y, 
                      eval_set=[(val_x, val_y)], 
                      callbacks=[log_cb, early_stop_cb])
            
            del trn_x
            gc.collect()
            preds += ((model.predict((val_x))) / execs)
            test_preds_exec += ((model.predict((test[train_columns]))) / execs)

        test_preds_fold += (test_preds_exec / n_folds)
        oof_preds[val_] = preds
        score_folds.append(np.sqrt(mean_squared_error(val_y, preds)))
        print("FOLD RMSE = {}".format(np.sqrt(mean_squared_error(val_y, preds))))
        del model, val_x
        gc.collect()

    print("############################################################")
    print("{} - MEAN RMSE = {}".format(target, np.mean(score_folds)))
    print("{} - OOF RMSE = {}".format(target, np.sqrt(mean_squared_error(y, oof_preds))))
    print(score_folds)

    if target == 'North':
        north_oof_preds = oof_preds
        north_test_preds = test_preds_fold
    else:
        east_oof_preds = oof_preds
        east_test_preds = test_preds_fold

Fold: 1.0
Iteration: 1
Training until validation scores don't improve for 200 rounds
[500]	valid_0's rmse: 0.26671
[1000]	valid_0's rmse: 0.258965
[1500]	valid_0's rmse: 0.254628
[2000]	valid_0's rmse: 0.251575
[2500]	valid_0's rmse: 0.250031
Early stopping, best iteration is:
[2300]	valid_0's rmse: 0.249777
FOLD RMSE = 0.249776586246013
Fold: 2.0
Iteration: 1
Training until validation scores don't improve for 200 rounds
[500]	valid_0's rmse: 0.317169
[1000]	valid_0's rmse: 0.311273
Early stopping, best iteration is:
[900]	valid_0's rmse: 0.30954
FOLD RMSE = 0.3095395670628598
Fold: 3.0
Iteration: 1
Training until validation scores don't improve for 200 rounds
[500]	valid_0's rmse: 0.274881
[1000]	valid_0's rmse: 0.267199
[1500]	valid_0's rmse: 0.262283
[2000]	valid_0's rmse: 0.258999
[2500]	valid_0's rmse: 0.257309
[3000]	valid_0's rmse: 0.254924
[3500]	valid_0's rmse: 0.253359
[4000]	valid_0's rmse: 0.252055
[4500]	valid_0's rmse: 0.250915
[5000]	valid_0's rmse: 0.250162
[5500]	valid

In [18]:
all_preds = np.concatenate([north_oof_preds, east_oof_preds])
all_target = np.concatenate([train.North, train.East])

In [19]:
print("ALL - OOF RMSE = {}".format(np.sqrt(mean_squared_error(all_target, all_preds))))

ALL - OOF RMSE = 0.28435407409411206


In [20]:
sub_oof = pd.DataFrame({
    'Filename': train.Filename,
    'oof_north': north_oof_preds,
    'oof_east': east_oof_preds
})

sub_oof.to_csv('oof_lgb_seq_003.csv', index=False)

In [21]:
sub_north_test = pd.DataFrame({
    'Id': test.Filename,
    'Predicted': north_test_preds
})

sub_east_test = pd.DataFrame({
    'Id': test.Filename,
    'Predicted': east_test_preds
})

In [22]:
sub_north_test.Id = sub_north_test['Id'].astype(str) + ':North'
sub_east_test.Id  = sub_east_test['Id'].astype(str) + ':East'

sub_all_test = pd.concat([sub_north_test, sub_east_test])

In [23]:
sub_all_test[['Id', 'Predicted']].to_csv('sub_lgb_seq_003.csv', index=False)

In [24]:
sub_all_test

,Id,Predicted
0,0f57a863af3b7e5bf59a94319a408ff7.jpg:North,-0.287038
1,b7d4e3a6acb3b956d8b09416958e8892.jpg:North,-0.442397
2,9a1bd630a47dabafa6f4cab7338d01df.jpg:North,-0.560369
3,3f6ebf8aef0f352720bc0cb534d878b6.jpg:North,-0.551456
4,7e810f05bd88ff0f86dce7116b9fa527.jpg:North,-0.675854
...,...,...
55026,fc65c23ae61ce2c6fdbc637a8296d1d7.jpg:East,1.459978
55027,fc6c0d99ec42bad5e2d5fbb1e71c5a50.jpg:East,1.514243
55028,fdbb732633c6a3d32a67fc56edc7cde4.jpg:East,-0.256744
55029,feac1b902c7487dcbdf298bd4be75690.jpg:East,-0.315537
